<a href="https://colab.research.google.com/github/Deep-of-Machine/AI_Academy/blob/main/5_4_mlp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## GPU로 학습
런타임 - 런타임 유형 변경 - GPU    
이후 신경망을 학습할 때는 전부 런타임 유형을 GPU를 사용합니다.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import time

import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from torch import nn
from torch.nn import functional as F
from torch import optim

In [ ]:
print(torch.cuda.is_available())

In [ ]:
device = torch.device('cuda')
print(device)

In [ ]:
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5), (0.5))])
train_dataset = datasets.MNIST(root = './data', train = True, download = True, transform = transform)
test_dataset = datasets.MNIST(root = './data', train = False, download = True, transform = transform)

In [ ]:
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

In [ ]:
class MyNet(nn.Module):
    def __init__(self):
        super(MyNet, self).__init__()
        self.layer1 = nn.Linear(784, 128)
        self.layer2 = nn.Linear(128, 32)
        self.layer3 = nn.Linear(32, 10)
    
    def forward(self, x):
        x = self.layer1(x)
        x = F.relu(x)
        x = self.layer2(x)
        x = F.relu(x)
        out = self.layer3(x)

        return out


.to(device)를 사용해서 모델, 손실 함수, 데이터를 GPU에 할당

In [ ]:
model = MyNet().to(device)

print(model)

In [ ]:
loss_func = nn.CrossEntropyLoss().to(device)
optimizer = optim.SGD(model.parameters(), lr = 0.001)

In [ ]:
epochs = 10

for epoch in range(epochs):
    running_loss = 0.0
    start_time = time.time()

    for inputs, labels in train_loader:
        inputs = inputs.view(inputs.shape[0], -1).to(device)
        labels = labels.to(device)
        outputs = model(inputs)

        loss = loss_func(outputs, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    end_time = time.time()

    print('Epoch : {}, Loss : {}, time : {}'.format(epoch + 1, running_loss/len(train_loader), end_time - start_time))


## 모델 저장과 불러오기
연동한 구글 드라이브 경로에 저장하면, 이 후에 학습된 모델을 불러와서 사용 가능

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
PATH = '/content/drive/MyDrive/mnist_mlp.pth'
torch.save(model.state_dict(), PATH)

model2를 새로 생성하고, model과 비교




In [ ]:
model2 = MyNet()
print(list(model.parameters())[1][:10])  # 위에서 학습한 model의 파라미터 일부
print(list(model2.parameters())[1][:10]) # 새로 생성한 model2의 파라미터 일부

PATH에 저장된 가중치를 model2에 로드

In [ ]:
model2.load_state_dict(torch.load(PATH))
print(list(model2.parameters())[1][:10])

## 학습 가능한 파라미터 수 계산

In [ ]:
model.parameters()

In [ ]:
for i in model.parameters():
    print(i)
    break

In [ ]:
print(i.requires_grad == True)

In [ ]:
print(i.shape)

In [ ]:
print(i.shape[0] * i.shape[1])

In [ ]:
print(i.numel())

In [ ]:
n = 0
for i in model.parameters():
    if i.requires_grad == True:
        n += i.numel()

print(n)

직접 계산

## 모델 평가

In [ ]:
for mini_batch in test_loader:
    first_batch = mini_batch
    break

inputs = first_batch[0]
labels = first_batch[1]
outputs = model(inputs.view(inputs.shape[0], -1).to(device))

In [ ]:
sample_number = 0
plt.imshow(inputs[sample_number].squeeze().numpy(), cmap = 'gray')
plt.show()
print('예측 : {}'.format(torch.argmax(outputs[sample_number]).item()))
print('라벨 : {}'.format(labels[sample_number].item()))

In [ ]:
correct = 0
total = 0

with torch.no_grad():
    for inputs, labels in test_loader:
        inputs = inputs.view(inputs.shape[0], -1).to(device)
        labels = labels.to(device)
        outputs = model(inputs)
        predicted = torch.argmax(outputs, dim = 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print(total)
    print(100*correct / total)